# 广播和向量化

In [12]:
using BenchmarkTools
using Plots

## 绘制二维曲面

In [1]:
f(x, y) = x * exp(-x^2-y^2)

f (generic function with 1 method)

## Julia里的广播
广播不会让Julia变得更快，它只是把for循环藏起来了。

In [4]:
g(x, y) = x + y

g (generic function with 1 method)

In [5]:
g.([1,2], [3,4])

2-element Vector{Int64}:
 4
 6

In [6]:
g.([1,2],3)

2-element Vector{Int64}:
 4
 5

In [17]:
X1 = reshape([1,2,3,4,3,4,5,6], (4, 2))
Y1 = [2,3,5,7]
g.(X1,Y1)

4×2 Matrix{Int64}:
  3   5
  5   7
  8  10
 11  13

在内存中的不同排列方式会影响代码的写法，进而影响计算效率。

In [19]:
x6 = reshape(1:12,2,3,2)

2×3×2 reshape(::UnitRange{Int64}, 2, 3, 2) with eltype Int64:
[:, :, 1] =
 1  3  5
 2  4  6

[:, :, 2] =
 7   9  11
 8  10  12

In [20]:
y6 = permutedims(x6, (3,1,2))

2×2×3 Array{Int64, 3}:
[:, :, 1] =
 1  2
 7  8

[:, :, 2] =
 3   4
 9  10

[:, :, 3] =
  5   6
 11  12

## 向量化

- f(x::Real, y::Real) = x * y
- f(X::AbstractArray, y::AbstractArray) = X .* Y

Julia不推荐向量化编程，这是因为：~~~

In [21]:
muladd

muladd (generic function with 21 methods)

In [22]:
muladd.([1,2,3],[2,3,4],[1,2,3])

3-element Vector{Int64}:
  3
  8
 15

In [23]:
muladd_vec(A::AbstractArray, B, C) = A .* B + C
muladd_temp(a::Number, b, c) = a * b + c
muladd_broadcast(A::AbstractArray, B, C) = muladd_temp.(A, B, C)

muladd_broadcast (generic function with 1 method)

In [26]:
A = rand(4,2)
B = rand(4,2)
C = rand(4,2)

@btime muladd.($A,$B,$C)
@btime muladd_vec($A,$B,$C)
@btime muladd_broadcast($A,$B,$C)

  61.633 ns (1 allocation: 144 bytes)
  122.591 ns (2 allocations: 288 bytes)
  66.904 ns (1 allocation: 144 bytes)


4×2 Matrix{Float64}:
 0.246729  0.847313
 0.261622  1.03784
 1.25908   0.225348
 0.373756  0.692663

上面的例子表明向量化不仅占用了更多内存，而且还慢了很多。

## View 和 Copy

In [28]:
X = [1,2,3,4]
Y = X[1:4]
Y[1] = 0
@show X,Y;

(X, Y) = ([1, 2, 3, 4], [0, 2, 3, 4])


In [29]:
X = [1,2,3,4]
Y = @view X[1:4]
Y[1] = 0
@show X,Y;

(X, Y) = ([0, 2, 3, 4], [0, 2, 3, 4])


## Ex. MNIST数据集

## 向量化对并行计算是很有用的

In [8]:
function g_kernel(x)
    sleep(0.01)
    return x
end

g_kernel (generic function with 1 method)

In [9]:
function g_seq(X)
    out = similar(X)
    for i in eachindex(X)
        out[i] = g_kernel(X[i])
    end
    return out
end

g_seq (generic function with 1 method)

In [10]:
function g_threads(X)
    out = similar(X)
    Threads.@threads for i in eachindex(X)
        out[i] = g_kernel(X[i])
    end
    return out
end

g_threads (generic function with 1 method)

In [13]:
X = rand(4,4)
@btime g_seq($X)
@btime g_threads($X)

  243.121 ms (85 allocations: 2.52 KiB)
  241.295 ms (91 allocations: 3.11 KiB)


4×4 Matrix{Float64}:
 0.406844  0.184595  0.448362   0.101233
 0.411754  0.895952  0.93337    0.0554428
 0.864226  0.717108  0.0495627  0.98938
 0.240794  0.482872  0.504993   0.908991